## Project title: 
## Box Office Blueprint: Data-Driven Insights for Smarter Movie Production

## COLLABORATORS
1. PAULINE KIMENZU
2. JORAM MUGESA
3. EDNA OGOTI
4. KELVIN SESERY

## BUSINESS PROBLEM
Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

## MAIN OBJECTIVES
To analyze current box office performance and identify the types of films that achieve the greatest financial success and audience appeal, providing actionable insights for the company’s new movie studio.

## SPECIFIC OBJECTIVES 
1. Are certain  movie directors more likely to have more revenue in the box office?

2. Which genres are more likely to get highest critics ratings? 

3. Is there a relationship between the production budget and revenue in worldwide gross? 

4. Which original languages are more popular in screening in box office?


## DATA
## DATA UNDERSTANDING
The project will use five distinct datasets to analyze movie performance and market trends:
1. bom.movie_gross.csv: This file contains box office data, including domestic and foreign gross revenue for movies.

2. tn.movie_budgets.csv: This dataset provides key financial information for movies, including production budget, domestic gross, and worldwide gross. It's crucial for analyzing profitability.

3. tmdb.movies.csv: This file includes a unique identifier for each movie, as well as genre IDs, popularity scores, and vote averages. The genre IDs will need to be mapped to their corresponding names.

4. rt.movie_info.tsv: This dataset contains descriptive information for each movie, such as genre, director, and synopses from Rotten Tomatoes.

5. rt.reviews.tsv: This file provides critical reviews and ratings for movies, which can be used to assess critical reception.

6. im.db dataset is an SQLite database.The database contains the following tables:
movie_basics – likely holds general information about movies (title, year, genre, etc.).
directors – probably links movies with directors.
known_for – could be movies that actors/directors are known for.
movie_akas – alternate titles (AKAs) for movies.
movie_ratings – ratings and votes data.
persons – information about actors, directors, writers, etc.
principals – probably links movies to main cast/crew.
writers – movie-to-writer mappings.

The primary goal of the  project is:
To provide data-driven insights that help the new film studio identify what types of movies are most likely to succeed at the box office, so the company can make smarter investment and production decisions.
in nutshell:  use data to reduce risk and increase the chances of producing profitable films.

## loading datasets

In [1]:
# Your code here - remember to use markdown cells for comments as well!
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sqlite3


## loading bom.movie_gross.csv dataset

In [2]:
#read the data and check the first five columns

df1 = pd.read_csv("zippedData/bom.movie_gross.csv.gz",low_memory=False)        

df1.head()
#excluded dataset

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


## loading tn.movie_budgets.csv dataset

In [3]:
# read the data and check the first five columns
df2 = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")        

df2.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## loading tmdb.movies.csv dataset

In [4]:
#read the data and check the first five columns
df3 = pd.read_csv("zippedData/tmdb.movies.csv.gz")  
df3.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


## loading rt.movie_info.tsv dataset

In [5]:
# read the data and check the first five columns
df4 = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t",compression="gzip",encoding="latin-1") 
df4.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


## loading rt.reviews.tsv dataset

In [6]:
#read the data and check the first five columns
df5 = pd.read_csv("zippedData/rt.reviews.tsv.gz", sep="\t",compression="gzip",encoding="latin-1") 
df5.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


## loading im.db dataset 

In [7]:
#establish connection to database
conn = sqlite3.connect("zippedData/im.db")
tables = pd.read_sql("SELECT name FROM sqlite_master", conn)
tables

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [8]:
df_movaka = pd.read_sql("SELECT * FROM movie_akas;", conn)
df_movaka

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [9]:
df_writer = pd.read_sql("SELECT * FROM movie_basics;", conn)
df_writer

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [10]:
df_movies = pd.read_sql("SELECT * FROM movie_basics;", conn)
df_movies

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [11]:
df_rating = pd.read_sql("SELECT * FROM movie_ratings;", conn)
df_rating

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [12]:
df_directors = pd.read_sql("SELECT * FROM directors;", conn)
df_directors

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502
...,...,...
291169,tt8999974,nm10122357
291170,tt9001390,nm6711477
291171,tt9001494,nm10123242
291172,tt9001494,nm10123248


## data cleaning,exploration and visualization


 ### Data Cleaning and Analysis 1. Are certain movie directors more likely to have more revenue in the box office?


In [13]:
# read the data and check the first five columns
df4 = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t",compression="gzip",encoding="latin-1") 
df4.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [14]:
df4.info

<bound method DataFrame.info of         id                                           synopsis rating  \
0        1  This gritty, fast-paced, and innovative police...      R   
1        3  New York City, not-too-distant-future: Eric Pa...      R   
2        5  Illeana Douglas delivers a superb performance ...      R   
3        6  Michael Douglas runs afoul of a treacherous su...      R   
4        7                                                NaN     NR   
...    ...                                                ...    ...   
1555  1996  Forget terrorists or hijackers -- there's a ha...      R   
1556  1997  The popular Saturday Night Live sketch was exp...     PG   
1557  1998  Based on a novel by Richard Powell, when the l...      G   
1558  1999  The Sandlot is a coming-of-age story about a g...     PG   
1559  2000  Suspended from the force, Paris cop Hubert is ...      R   

                                                  genre            director  \
0                   Acti

In [57]:
# Count of nulls per column
df4.isnull().sum()

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [58]:
# Dropped director and box-office and write
df4_cleaned = df4.dropna(subset=["director","box_office","writer"])
df4_cleaned.isnull().sum()

id               0
synopsis         0
rating           0
genre            0
director         0
writer           0
theater_date     3
dvd_date         3
currency         0
box_office       0
runtime          2
studio          26
dtype: int64

In [59]:
df4_cleaned.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,600000.0,108 minutes,Entertainment One
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,"Jan 11, 2002","Jun 18, 2002",$,41032915.0,82 minutes,Paramount Pictures
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,"Apr 27, 2006","Oct 2, 2007",$,224114.0,123 minutes,Sony Pictures Classics
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,"Jun 30, 2010","Nov 9, 2010",$,134904.0,117 minutes,NaN
15,22,Two-time Academy Award Winner Kevin Spacey giv...,R,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,"Dec 17, 2010","Apr 5, 2011",$,1039869.0,108 minutes,ATO Pictures


In [60]:
#Check the data types available 
print(df4_cleaned.dtypes)

id                int64
synopsis         object
rating           object
genre            object
director         object
writer           object
theater_date     object
dvd_date         object
currency         object
box_office      float64
runtime          object
studio           object
dtype: object


In [61]:
#Convert the data type to float
df4["box_office"] = pd.to_numeric(
    df4["box_office"].replace({r'\$': '', ',': ''}, regex=True),
    errors="coerce"
)

top_directors = df4_cleaned.groupby("director")["box_office"].sum().sort_values(ascending=False).head(20)
top_directors


In [67]:
# Explore the directors with the highest box offices
top_directors = df4_cleaned.groupby(["director","genre"])["box_office"].sum().sort_values(ascending=False).head(20)
top_directors

director                     genre                                                                
Jay Roach                    Comedy                                                                   443622410.0
Mel Gibson                   Drama                                                                    368000000.0
Peter Jackson                Action and Adventure|Science Fiction and Fantasy                         303001229.0
Sam Mendes                   Action and Adventure|Mystery and Suspense                                299300000.0
Chris Columbus               Kids and Family|Science Fiction and Fantasy                              261835892.0
Joel Zwick                   Comedy|Romance                                                           241250669.0
Steven Spielberg             Action and Adventure|Science Fiction and Fantasy                         234141872.0
Peter Berg                   Action and Adventure|Comedy|Drama                                        2

 ### Objective 2: Which genres are more likely to get highest critics ratings? 

In [76]:
df_rating = pd.read_sql("SELECT * FROM movie_ratings;", conn)
df_rating

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [82]:
#Check missing Values  on the Review Dataset

# Count of nulls per column
df_rating_clean = df_rating.isna().sum()
df_rating


,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [72]:
# Explore Data on the Movie ratings data with the movie basics
query = """SELECT movie_id , averagerating, numvotes, genres 
FROM movieratings JOIN

SyntaxError: invalid syntax (1857393311.py, line 2)

In [101]:
query ="""SELECT 
    b.genres,
    r.movie_id,
    AVG(r.averagerating) AS avg_rating,
    SUM(r.numvotes) AS total_votes
FROM  movie_ratings r
JOIN movie_basics b
    ON r.movie_id = b.movie_id
    GROUP BY b.genres
    ORDER BY avg_rating DESC;"""
df_genre = pd.read_sql(query,conn)
df_genre

,genres,movie_id,avg_rating,total_votes
0,"Comedy,Documentary,Fantasy",tt4135932,9.4,5
1,"Documentary,Family,Musical",tt3856476,9.3,19
2,"History,Sport",tt5903964,9.2,5
3,"Music,Mystery",tt1954785,9.0,5
4,Game-Show,tt2896176,9.0,7
...,...,...,...,...
919,"Crime,Music",tt8463476,2.4,88
920,"History,Sci-Fi,Thriller",tt4656810,2.3,227
921,"Adventure,Crime,Romance",tt3140634,2.3,9
922,"Adult,Horror",tt3718824,2.0,128
